In [90]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
import requests
from io import StringIO
import re
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import umap.umap_ as umap
import matplotlib.pyplot as plt
import hdbscan
pd.set_option('display.max_colwidth', None)

In [91]:
sheet_name_lsit = ['Sci/2', 'Sci_Extra/1', 'Sci/1', 'Sci_Extra/2', 'Engineering/1', 'Engineering_Extra/1', 'Engineering/2', 'Engineering_Extra/2', 'SIIT/2', 'SIIT/1', 'Innovation_Extra/1', 'Innovation_Extra/2', 'CIS_Extra/1', 'CIS/1', 'CIS_Extra/2', 'CIS/2']

In [92]:
sheet_id = "1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU"
dataframe = pd.DataFrame(columns=["code", "desc", "add1", "add2", "add3","ma"])

for sheet_name in sheet_name_lsit:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    response = requests.get(url)
    decoded_content = response.content.decode('utf-8')
    data_tem =  pd.read_csv(StringIO(decoded_content))
    dataframe = pd.concat([dataframe, data_tem], ignore_index=True)

print(dataframe.shape)
for i in dataframe.columns:
    dataframe[f"{i}"] = dataframe[f"{i}"].str.strip().str.lower()


print(dataframe.shape)

(4272, 6)
(4272, 6)


In [93]:
dataframe["desc"] = dataframe["desc"].str.replace(r"[.,]", "", regex=True).str.strip()

dataframe.drop_duplicates(subset=["code"],inplace=True)
dataframe.dropna(subset=["desc","code"],inplace=True)
df = dataframe[dataframe.duplicated(subset=["desc"], keep=False)]
print(df.desc.head(3))
print(df.shape)
dataframe.drop_duplicates(subset=["desc"], keep=False,inplace=True)

df["code"] = df.groupby("desc")["code"].transform(lambda x: ", ".join(x))
df = df.drop_duplicates(subset="desc")
print(df.shape)
dataframe = pd.concat([df,dataframe])

19     discussion on topics of current interest in applied mathematics basic skills for academic reading proposal topic regulations and procedures literature review designing and planning of the project report writing and topic of project presentation
59                                   biology of animal behaviour behavioural genetics memory and learning foraging and ingestive behaviour social and reproductive behaviour farm animal behaviour regulations and assessment of animal welfare field trips
213                                          senior projects will be assigned or approved by the department this course prepares students for developing a proposal for doing research writing research papers and presenting the proposal to the committee
Name: desc, dtype: object
(169, 6)
(83, 6)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11748\3070791526.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["code"] = df.groupby("desc")["code"].transform(lambda x: ", ".join(x))


In [94]:
df.head(10)

,code,desc,add1,add2,add3,ma
19,"am360, ma391",discussion on topics of current interest in applied mathematics basic skills for academic reading proposal topic regulations and procedures literature review designing and planning of the project report writing and topic of project presentation,['disciplinary course(s): am260\nสอบได้ คป.260 หรือ ได้รับอนุมัติจากผู้บรรยาย'],['consecutive course(s): am460'],[],sci
59,"at437, at438",biology of animal behaviour behavioural genetics memory and learning foraging and ingestive behaviour social and reproductive behaviour farm animal behaviour regulations and assessment of animal welfare field trips,[],[],[],sci
213,"cs303, cs401",senior projects will be assigned or approved by the department this course prepares students for developing a proposal for doing research writing research papers and presenting the proposal to the committee,['disciplinary course(s): cs300 and cs301 or\ncs300 and cs302\nสำหรับนักศึกษาตั้งแต่ชั้นปีที่ 3 ขึ้นไป'],['consecutive course(s): cs403'],[],sci
235,"cs369, cs485",basic systems and protocols for providing services on the internet developing and deploying web applications topics covered web application development techniques for both client side and server side session management interfacing with other online services web application security web design patterns and reusable web application components,['disciplinary course(s): cs234 or\ncs322'],[],[],sci
238,"cs379, cs466",deterministic and stochastic grammars morphological analysis syntax analysis semantic analysis discourse analysis and applications (eg machine translation speech recognition and synthesis and text mining),['disciplinary course(s): cs265'],[],[],sci
255,"es201, es391",integration of principles and theories to be applied to systematically solve complex problems in the environment to distinguish synthesize and analyze information from world current online information in order to properly present the facts to the public in accordance with environmentalist ethics,[],[],[],sci
429,"np190, np391",challenge problems are given to students with the objective of enhancing and creating a road map of developing creative and innovative thoughts in their minds main focus is upon international current developments and trends of innovation in the field of applied physics or electronics with the assistance of challenge problems students can gain the methodology of conceptual learning by themselves and crystalize their ideas for the suggestion of the problems’ solutions to committee grades: s or u,[],[],[],sci
430,"np200, np210",elementary circuit theory analysis of dc and ac circuits frequency filter circuits semiconductors and p-n junction diode circuits transistors amplifier circuits digital systems elementary principles of logic gates flip-flops registers counting circuits digital communications,['disciplinary course(s): sc132 or\nparallel degree/ dual degree sc132 or\nsc135'],"['consecutive course(s): np210, np211, np217, np220']",[],sci
447,"pc101, pc203",computational tools to do data analysis solve mathematics and physics problems and visualize the data and results spreadsheet software computer algebra systems computer programming and debugging,['consecutive course(s): pc346'],[],[],sci
666,"tg323, tg326",principles and procedures for chemical and physical finishing nature of chemical substance used in textile finishing process to impart specific end-use properties new techniques for finishing of textile products and applications field trips are included,['disciplinary course(s): tg322'],[],[],sci


In [95]:
dataframe["add1"] = dataframe["add1"].str.replace("คพ","cs")
dataframe["add1"] = dataframe["add1"].str.replace("ทก","at")
dataframe["add1"] = dataframe["add1"].str.replace("คป","am")
dataframe["add1"] = dataframe["add1"].str.replace("ทช","bt")
dataframe["add1"] = dataframe["add1"].str.replace("วท","sc")
dataframe["add1"] = dataframe["add1"].str.replace("คม","cm")
dataframe["add1"] = dataframe["add1"].str.replace("ค","ma")
dataframe["add1"] = dataframe["add1"].str.replace("ส","st")

dataframe["add2"] = dataframe["add2"].str.replace("คพ","cs")
dataframe["add2"] = dataframe["add2"].str.replace("ทก","at")
dataframe["add2"] = dataframe["add2"].str.replace("คป","am")
dataframe["add2"] = dataframe["add2"].str.replace("ทช","bt")
dataframe["add2"] = dataframe["add2"].str.replace("วท","sc")
dataframe["add2"] = dataframe["add2"].str.replace("คม","cm")
dataframe["add2"] = dataframe["add2"].str.replace("ค","ma")
dataframe["add2"] = dataframe["add2"].str.replace("ส","st")


In [96]:
def extract_courses(row):
    pattern = r"[A-Za-zก-ฮ]{1,2}\.? ?\d{3}"
    listt = []
    if isinstance(row, str):
        matches = re.findall(pattern, row)  
        listt.extend(matches)
        return listt
    return None 

dataframe["cancel_course"] = dataframe["add1"].apply(extract_courses) + dataframe["add2"].apply(extract_courses)

In [97]:
dataframe.drop("add3",inplace=True,axis=1)

In [101]:
dataframe.to_csv("data1.csv")

In [100]:
dataframe.rename(columns={
    "desc":"description",
    "add1":"disciplinary course(s)",
    "add2":"consecutive course(s)",
    "ma":"faculty"
    
},inplace=True)